In [1]:
import pandas as pd

ADMISSION_DATA_COLS = ["subject_id", "hadm_id", "marital_status", "race"]
PERSONAL_DATA_COLS = ["subject_id", "gender", "anchor_age"]
MESURE_DATA_COLS = ['hadm_id', 'stay_id', 'charttime', "itemid", "valuenum"]

In [2]:
from constants import TEMP_PATH


dfPatient = pd.read_csv(TEMP_PATH / "icu_target.csv", usecols=["hadm_id", "stay_id"])
dfPatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8866 entries, 0 to 8865
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   hadm_id  8866 non-null   int64
 1   stay_id  8866 non-null   int64
dtypes: int64(2)
memory usage: 138.7 KB


In [3]:
from constants import MIMIC_PATH


dfPatientAdmission = pd.read_csv(MIMIC_PATH / "hosp" / "admissions.csv", usecols=ADMISSION_DATA_COLS)
dfPatientPersonal = pd.read_csv(MIMIC_PATH / "hosp" / "patients.csv", usecols=PERSONAL_DATA_COLS)

dfPatientInfo = pd.merge(dfPatientAdmission, dfPatientPersonal, on="subject_id")
del dfPatientAdmission
del dfPatientPersonal

dfPatientInfo.drop(columns="subject_id", inplace=True)
dfPatientInfo["marital_status"] = dfPatientInfo["marital_status"].astype("category")
dfPatientInfo["race"] = dfPatientInfo["race"].astype("category")
dfPatientInfo["gender"] = dfPatientInfo["gender"].astype("category")

dfPatientInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431231 entries, 0 to 431230
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   hadm_id         431231 non-null  int64   
 1   marital_status  421998 non-null  category
 2   race            431231 non-null  category
 3   gender          431231 non-null  category
 4   anchor_age      431231 non-null  int64   
dtypes: category(3), int64(2)
memory usage: 7.8 MB


In [4]:
dfPatient = pd.merge(dfPatient, dfPatientInfo, on= "hadm_id")
del dfPatientInfo

dfPatient.drop(columns="hadm_id", inplace=True)
dfPatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8866 entries, 0 to 8865
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   stay_id         8866 non-null   int64   
 1   marital_status  8293 non-null   category
 2   race            8866 non-null   category
 3   gender          8866 non-null   category
 4   anchor_age      8866 non-null   int64   
dtypes: category(3), int64(2)
memory usage: 166.3 KB


In [5]:
dfIcuStay = pd.read_csv(
    MIMIC_PATH / "icu" / "icustays.csv", 
    usecols=["stay_id", "intime", "outtime"], 
    parse_dates=["intime", "outtime"]
)
dfPatient = pd.merge(dfPatient, dfIcuStay, on= "stay_id")
del dfIcuStay

dfPatient.to_csv(TEMP_PATH / "dfPatient.csv")
dfPatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8866 entries, 0 to 8865
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   stay_id         8866 non-null   int64         
 1   marital_status  8293 non-null   category      
 2   race            8866 non-null   category      
 3   gender          8866 non-null   category      
 4   anchor_age      8866 non-null   int64         
 5   intime          8866 non-null   datetime64[ns]
 6   outtime         8866 non-null   datetime64[ns]
dtypes: category(3), datetime64[ns](2), int64(2)
memory usage: 304.8 KB


In [22]:
from merge_data import mergeData


MESURE_INTERVAL = pd.Timedelta(hours=6)
MESURE_GROUP_LENGTH = 6

mergeData(dfPatient, MESURE_INTERVAL, MESURE_GROUP_LENGTH)

: 